In [1]:
!pip3 install pandas spacy tqdm

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.1/12.8 MB 919.0 kB/s eta 0:00:14
     - -------------------------------------- 0.4/12.8 MB 3.2 MB/s eta 0:00:04
     ----- ---------------------------------- 1.7/12.8 MB 9.8 MB/s eta 0:00:02
     ------------ --------------------------- 4.1/12.8 MB 18.5 MB/s eta 0:00:01
     ------------------ --------------------- 5.8/12.8 MB 21.6 MB/s eta 0:00:01
     ----------------------- ---------------- 7.5/12.8 MB 24.0 MB/s eta 0:00:01
     ---------------------------- ----------- 9.2/12.8 MB 25.7 MB/s eta 0:00:01
     --------------------------------- ----- 11.0/12.8 MB 38.6 MB/s eta 0:00:01
     --------------------------------------  12.7/12.8 MB 38.5 MB/s eta 0:00:01
     --------------------------------------  12.8/12.8 MB 38

In [3]:
!python -m spacy download es_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------------------------------------- 0.1/12.9 MB 656.4 kB/s eta 0:00:20
      --------------------------------------- 0.3/12.9 MB 2.0 MB/s eta 0:00:07
     --- ------------------------------------ 1.1/12.9 MB 6.3 MB/s eta 0:00:02
     ---------- ----------------------------- 3.5/12.9 MB 16.0 MB/s eta 0:00:01
     ---------------- ----------------------- 5.3/12.9 MB 19.7 MB/s eta 0:00:01
     --------------------- ------------------ 7.1/12.9 MB 21.5 MB/s eta 0:00:01
     --------------------------- ------------ 8.8/12.9 MB 24.5 MB/s eta 0:00:01
     ------------------------------- ------- 10.6/12.9 MB 38.6 MB/s eta 0:00:01
     ------------------------------------- - 12.3/12.9 MB 38.5 MB/s eta 0:00:01
     --------------------------------------  12.9/12.9 MB 38

In [1]:
import re

# Importar las librerías necesarias
import pandas as pd
import spacy
from tqdm.auto import tqdm

tqdm.pandas()
nlp = spacy.load('en_core_web_sm')

In [2]:
# Load the dataset
df = pd.read_csv('../data/cyberbullying.csv')
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",0
1,Why is #aussietv so white? #MKR #theblock #ImA...,0
2,@XochitlSuckkks a classy whore? Or more red ve...,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",0
4,@RudhoeEnglish This is an ISIS account pretend...,0


In [3]:
# Imprimir la stops words del spacy
print('Un total de:', len(nlp.Defaults.stop_words), 'stop words.')
print(nlp.Defaults.stop_words)
# 

Un total de: 326 stop words.
{'move', 'something', 'somehow', 'if', 'am', 'by', 'see', 'elsewhere', 'themselves', 'thence', 'just', 'take', 'done', 'not', '‘ve', 'who', 'further', 'unless', 'get', 'various', 'through', 'becoming', 'onto', 'will', 'must', 'even', 'can', 'please', 'an', 'after', 'do', 'during', 'hereby', 'eight', 'neither', 'she', 'call', 'top', '’d', 'besides', 'everything', 'or', 'few', 'whatever', 'already', 'could', 'anyway', 'thereby', 'quite', 'for', 'fifteen', 'became', 'behind', 'somewhere', "n't", 'out', 'used', 'often', 'herein', 'once', 'several', '’re', 'their', 'most', 'my', 'itself', 'been', 'put', 'two', 'give', 'therein', 'the', "'m", 'five', 'nevertheless', 'did', 'when', 'forty', 'such', 'was', 'a', 'nobody', '‘d', 'one', 'n‘t', 'also', 'hence', 'many', 'then', 'how', 'beforehand', 'same', 'meanwhile', 'regarding', 'about', 'which', 'made', 're', 'first', 'its', 'from', 'formerly', 'across', 'full', '‘s', 'whither', 'hereafter', 'between', 'you', 'every

In [1]:
# Pre-compilar la expresión regular fuera de la función
# Patron sencillo para identificar URLs en un texto.
# url_pattern = re.compile(r'https?:\/\/\S+|www\.\S+')
# Patron para identificar URLs, hasta mal escritas, en un texto. (https://regex101.com/r/sHFkhh/2)
url_pattern = re.compile(
    r"""(?:https?:\/\/)?(?:www\.)?(?:[a-zA-Z0-9-]+\.[a-zA-Z]{2,6})(?:[-a-zA-Z0-9@:%_\+.~#?&\/\/=]*)""")
# Menciones, RTs (retweets)
clean_pattern = re.compile(r'@\S+|([^A-Za-z \t])|(\w+:\/\/\S+)|^rt')


def preprocess_text(text) -> list:
    text = text.lower()
    text = re.sub(url_pattern, '', text)
    text = re.sub(clean_pattern, '', text)

    doc = nlp(text)
    return [token for token in doc if not token.is_punct and not token.is_space]


def remove_stopwords(tokens) -> list:
    return [token for token in tokens if not token.is_stop]


def lemmatize(tokens) -> list:
    return [token.lemma_ for token in tokens]


def clean_text(text) -> str:
    tokens = preprocess_text(text)
    filtered_tokens = remove_stopwords(tokens)
    lemmatized_tokens = lemmatize(filtered_tokens)
    cleaned_text = ' '.join(lemmatized_tokens)
    return cleaned_text


NameError: name 're' is not defined

In [ ]:
# Seleccionar algunas filas para probar la función
df_sample = df.head(25).copy()
df_sample['tweet_preprocessed'] = df_sample['tweet_text'].apply(clean_text)

# Eliminar las filas vacías
df_sample.dropna(inplace=True)
df_sample = df_sample[df_sample['tweet_preprocessed'] != '']

print(df_sample)

In [54]:
# Copiar el DataFrame
df_sample = df.copy()

# Aplicar la función 'normalize' a los tweets seleccionados
df_sample['tweet_preprocessed'] = df_sample['tweet_text'].progress_apply(clean_text)

# Eliminar las filas vacías
df_sample.dropna(inplace=True)
df_sample = df_sample[df_sample['tweet_preprocessed'] != '']

# Imprimir los tweets pre-procesados
df_sample

  0%|          | 0/81417 [00:00<?, ?it/s]

,tweet_text,cyberbullying_type,tweet_preprocessed
0,"In other words #katandandre, your food was cra...",0,word katandandre food crapilicious mkr
1,Why is #aussietv so white? #MKR #theblock #ImA...,0,aussietv white mkr theblock imacelebrityau tod...
2,@XochitlSuckkks a classy whore? Or more red ve...,0,classy whore red velvet cupcake
3,"@Jason_Gio meh. :P thanks for the heads up, b...",0,meh p thank head concerned angry dude twitter
4,@RudhoeEnglish This is an ISIS account pretend...,0,isis account pretend kurdish account like isla...
...,...,...,...
81412,Happy birthday @paul_lander!,0,happy birthday
81413,RT @MilradJalen: @raesanni I agree it’s awful ...,0,agree awful make sense regardless mistake not ...
81414,I was called in yesterday at the Guidance Coun...,0,call yesterday guidance counselor office think...
81415,@Jeicky_the_cat @skwi69 Thanks,0,thank


In [55]:
df_sample.to_csv('../data/cyberbullying_preprocessed.csv', index=False)
df_sample


,tweet_text,cyberbullying_type,tweet_preprocessed
0,"In other words #katandandre, your food was cra...",0,word katandandre food crapilicious mkr
1,Why is #aussietv so white? #MKR #theblock #ImA...,0,aussietv white mkr theblock imacelebrityau tod...
2,@XochitlSuckkks a classy whore? Or more red ve...,0,classy whore red velvet cupcake
3,"@Jason_Gio meh. :P thanks for the heads up, b...",0,meh p thank head concerned angry dude twitter
4,@RudhoeEnglish This is an ISIS account pretend...,0,isis account pretend kurdish account like isla...
...,...,...,...
81412,Happy birthday @paul_lander!,0,happy birthday
81413,RT @MilradJalen: @raesanni I agree it’s awful ...,0,agree awful make sense regardless mistake not ...
81414,I was called in yesterday at the Guidance Coun...,0,call yesterday guidance counselor office think...
81415,@Jeicky_the_cat @skwi69 Thanks,0,thank
